### Week 2 - Ridge Regression

In [183]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import statsmodels.api as sm
import seaborn as sns

In [184]:
# load dataset from .csv, 
# split into X and y and  

dataset = pd.read_csv('kc_house_data.csv')
y = dataset[['price']]
X = dataset.drop(['price', 'id'],axis=1)

### Feature Engineering


In [185]:
X['house_age'] = X['date'].str.slice(0, 4).astype(int) -  X['yr_built']
X = X.drop(['date','yr_built'],axis=1)

In [186]:
X['house_age'].groupby(X['house_age']).count()

house_age
-1       12
 0      430
 1      285
 2      174
 3      165
       ... 
 111     50
 112     33
 113     28
 114     69
 115     26
Name: house_age, Length: 117, dtype: int64

Create dummies for categorical data:

In [187]:
categorical_features = ['waterfront', 'view','condition','grade', 'zipcode']
X_dummies =  pd.get_dummies(X, columns = categorical_features, prefix = categorical_features, drop_first=True)
X_dummies.shape

(21613, 102)

### Model function

In [188]:
def evaluate(model, input_train, input_test, output_train, output_test):
    model.fit(input_train, output_train)

    train_score = model.score(input_train,output_train)
    train_preds = model.predict(input_train)
    train_rmse = np.sqrt(mean_squared_error(output_train,train_preds))
    print("Train :: R^2 = " + "%.2f"%train_score + " :: RMSE = " + f'{train_rmse:,.0f}')

    test_score = model.score(input_test,output_test)
    test_preds = model.predict(input_test)
    test_rmse = np.sqrt(mean_squared_error(output_test,test_preds))
    print("Test  :: R^2 = " + "%.2f"%test_score  + " :: RMSE = " + f'{test_rmse:,.0f}')
    return model

### Best model from last week :: Linear Regression
This version includes zipcode dummies and house_age as suggested by our study group


In [189]:
x_train, x_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.2)

In [190]:
lr = LinearRegression()
evaluate(lr, x_train, x_test, y_train, y_test)


Train :: R^2 = 0.83 :: RMSE = 147,332
Test  :: R^2 = 0.84 :: RMSE = 155,045


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Ridge Regression

In [191]:
sscale = StandardScaler()
X_train_scale = sscale.fit_transform(x_train)
X_test_scale = sscale.transform(x_test)
y_train_scale = y_train - np.mean(y_train)
y_test_scale = y_test - np.mean(y_train)

a = np.geomspace(1,1000,100)
ridge_cv = RidgeCV(alphas=a, cv=5)

#not centralising y (Jacob)
model = evaluate(ridge_cv,X_train_scale,X_test_scale,y_train, y_test)
print("original y :: best lambda:" + "%.2f"%model.alpha_ + "\n")

#centralising y (Carleton)
model = evaluate(ridge_cv,X_train_scale,X_test_scale,y_train_scale, y_test_scale)
print("centralised y :: best lambda:" + "%.2f"%model.alpha_ + "\n")

Train :: R^2 = 0.83 :: RMSE = 147,334
Test  :: R^2 = 0.84 :: RMSE = 155,012
original y :: best lambda:3.76

Train :: R^2 = 0.83 :: RMSE = 147,334
Test  :: R^2 = 0.84 :: RMSE = 155,012
centralised y :: best lambda:3.76



### Investigation
I've noticed that for random samples x_train, x_test, I have different best alphas.
See below:

In [192]:
for n in range(3):
    print("-------   Iteration #", n, "   -------")
    x_train, x_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.2)
    
    sscale = StandardScaler()
    X_train_scale = sscale.fit_transform(x_train)
    X_test_scale = sscale.transform(x_test)
    y_train_scale = y_train - np.mean(y_train)
    y_test_scale = y_test - np.mean(y_train)

    a = np.geomspace(1,500,50)
    ridge_cv = RidgeCV(alphas=a, cv=5)

    #not centralising y (Jacob)
    model = evaluate(ridge_cv,X_train_scale,X_test_scale,y_train, y_test)
    print("original y ::best lambda:" + "%.2f"%model.alpha_ + "\n")

    '''
    #centralising y (Carleton)
    model = evaluate(ridge_cv,X_train_scale,X_test_scale,y_train_scale, y_test_scale)
    print("centralised y :: best lambda:" + "%.2f"%model.alpha_ + "\n")
    '''

-------   Iteration # 0    -------
Train :: R^2 = 0.84 :: RMSE = 149,496
Test  :: R^2 = 0.83 :: RMSE = 146,725
original y ::best lambda:2.76

-------   Iteration # 1    -------
Train :: R^2 = 0.84 :: RMSE = 149,651
Test  :: R^2 = 0.83 :: RMSE = 146,178
original y ::best lambda:7.61

-------   Iteration # 2    -------
Train :: R^2 = 0.84 :: RMSE = 148,041
Test  :: R^2 = 0.83 :: RMSE = 153,696
original y ::best lambda:1.66



##### Conclusion

Ridge regression didn't improve the model. That might explain why we have different best alphas for each sample: they are irrelevant.

I was already expecting no improvement, since Linear Regression model didn't show overfitting.

### Overfitting Example:

For the sake of learning, I need a model with overfitting, so I'm reducing the number of rows.


In [193]:
X_small = X_dummies[:500]
y_small = y[:500]
sscale_small = StandardScaler()
X_small_scale = sscale_small.fit_transform(X_small)


In [194]:
x_train, x_test, y_train, y_test = train_test_split(X_small, y_small, test_size=0.2, random_state=2)


In [195]:
lr = LinearRegression()
evaluate(lr, x_train, x_test, y_train, y_test)

Train :: R^2 = 0.89 :: RMSE = 116,926
Test  :: R^2 = 0.35 :: RMSE = 287,987


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

##### Overfitted

Finally, Train score better than Test score. We do have overfitting!

I can confirm this, running a cross validation over the whole data.


In [196]:
l = cross_validate(lr, X_small, y_small, cv=5, return_train_score=True, scoring='neg_root_mean_squared_error')
train_score = -np.mean(l['train_score'])
test_score = -np.mean(l['test_score'])
print("Train:", f'{train_score:,.0f}',"   Test:", f'{test_score:,.0f}')

Train: 118,425    Test: 368,157


######  Finding best alpha

In [197]:
pipe = Pipeline([ ('scale', StandardScaler()), ('ridge', Ridge()) ])
a = np.geomspace(100,200,500)
params = {'ridge__alpha' : a}
grid = GridSearchCV(pipe, param_grid=params, cv=5, scoring='neg_root_mean_squared_error')

In [198]:
grid.fit(x_train,y_train)
train_score = grid.score(x_train,y_train)
print("Train :: RMSE = " + f'{-train_score:,.0f}')

test_score = grid.score(x_test,y_test)
print("Test  :: RMSE = " + f'{-test_score:,.0f}')

Train :: RMSE = 131,831
Test  :: RMSE = 184,661


In [199]:
grid.best_params_

{'ridge__alpha': 126.45966954704173}

######  Applying best alpha manually to double-check

In [200]:
sscale = StandardScaler()
X_train_scale = sscale.fit_transform(x_train)
X_test_scale = sscale.transform(x_test)


ridge = Ridge(alpha=126)
evaluate(ridge,X_train_scale,X_test_scale,y_train, y_test)


Train :: R^2 = 0.86 :: RMSE = 131,785
Test  :: R^2 = 0.73 :: RMSE = 184,669


Ridge(alpha=126, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

# Final conclusion:
Ridge regression improved the model for the toy-example of 500 observations.

Linear Regression scores:

    Train :: R^2 = 0.89 :: RMSE = 116,926
    Test  :: R^2 = 0.35 :: RMSE = 287,987
    
    Cross Validation scores:
    Train: 118,425    
    Test: 368,157
    
Ridge Score (alpha = 126):

    Train :: R^2 = 0.86 :: RMSE = 131,785
    Test  :: R^2 = 0.73 :: RMSE = 184,669
    

